# Begin with Credentials and Connection to Trino

In [ ]:
import os
import pathlib
from dotenv import load_dotenv

# Load some standard environment variables from a dot-env file, if it exists.
# If no such file can be found, does not fail, and so allows these environment vars to
# be populated in some other way
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

Set session variable CATALOG to make query terms much more compact

In [ ]:
import trino
from sqlalchemy.engine import create_engine

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ['TRINO_USER'],
    host = os.environ['TRINO_HOST'],
    port = os.environ['TRINO_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

import pandas as pd

In [ ]:
cleanup = False

if cleanup:
    qres = engine.execute(f'show tables in ghgrp_demo')
    l = qres.fetchall()

    for schema in [ 'ghgrp_demo' ]:
        print(schema)
        qres = engine.execute(f'show tables in {schema}')
        l = qres.fetchall()

        for table in l:
            qres = engine.execute(f'drop table {schema}.{table[0]}')
            display(qres.fetchall())

        qres = engine.execute(f'show tables in {schema}')
        display(qres.fetchall())

        qres = engine.execute(f'drop schema {schema}')
        display(qres.fetchall())


    qres = engine.execute('show schemas')
    qres.fetchall()
    
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'ghgrp_demo'



# Introduction to EPA GHG Reporting Program data (EPA_GHGRP)

The EPA's GHG Reporting Program (GHGRP) seems to be a gold standard in terms of creating a bottoms-up list that's good enough to play a major role in tops-down estimates.

In [ ]:
qres = engine.execute(f"show tables in epa_ghgrp")
display(qres.fetchall())

`Direct_Emitters` are the lion's share of CO2 _emissions_.  `Suppliers` tracks fuels and products which, when used as intended, will create GHG emissions (by direct emitters or others).

In [ ]:
qres = engine.execute(f"describe epa_ghgrp.direct_emitters")
display(qres.fetchall())

In [ ]:
qres = engine.execute(f"""
select format('%tY', year), format('%,.2f', sum(total_reported_direct_emissions)/1000000000) || ' GtCO2e'
from epa_ghgrp.direct_emitters group by year order by year desc""")
display(qres.fetchall())

Here's a look at how they stack up (from a Database perspective--we should also look at this in Super Set).

In [ ]:
qres = engine.execute(f"""
select count (*), latest_reported_industry_type_sectors, format('%,.2f', sum(total_reported_direct_emissions)/1000000) || ' MtCO2e' as MtCO2e
from epa_ghgrp.direct_emitters
where year>=DATE('2019-01-01') and year<DATE('2020-01-01')
group by latest_reported_industry_type_sectors
order by MtCO2e desc
""")
display(qres.fetchall())

This looks at the `Minerals` industry (which includes cement).  We see that the top emitters have multiple facility locations.

In [ ]:
qres = engine.execute(f"""
select count (*), parent_company_name, format('%5.2f', sum(total_reported_direct_emissions)/1000000) || ' MtCO2e' as MtCO2e
from epa_ghgrp.direct_emitters, epa_ghgrp.parent_company
where year>=DATE('2019-01-01') and year<DATE('2020-01-01') and year=reporting_year
      and latest_reported_industry_type_sectors='Minerals'
      and epa_ghgrp.direct_emitters.facility_id=epa_ghgrp.parent_company.ghgrp_facility_id
group by parent_company_name
order by MtCO2e desc
limit 20
""")
display(qres.fetchall())

`Suppliers` are those who buy and sell GHG-emitting products, but they do not, themselves, cause the emissions.  They merely enable others to emit.

In [ ]:
qres = engine.execute(f"describe epa_ghgrp.suppliers")
display(qres.fetchall())

A quick summary of how many rows of data we have in `epa_ghgrp`.

68k rows in `direct_emitters`: lots of facilities  
103k rows in `parent_company`: lots of facility/owner relationships

In [ ]:
qres = engine.execute('show tables in epa_ghgrp')
l = qres.fetchall()
totalrows = 0
for e in l:
    s = f'select count (*) from epa_ghgrp.{e[0]}'
    qres = engine.execute(s)
    rowcount = qres.fetchall()[0][0]
    totalrows += rowcount
    print(f"{rowcount:>6} <- {s})")

print(f'{totalrows} <- total rows')

# Reshaping tables to make them easier to chart

The key metric is total_emissions (in metric tons of CO2e), but the name of the metric depends on the source/process.  Nevertheless, we know that `year` is our last metric and that the CO2e metric is 2nd-to-last (hence the `-2` index).

We also know that when building our final summary table, the sums feeding into it are all only one row per year.  We use `iat[0,1` to access the 0th row and the 1st column (which will be named specifically to the source/process).  By using `iat`, we get a scalar value we can sum, instead of a Series object we'd have to `squeeze`.

In [ ]:
import pandas as pd

emission_tables = ['direct_emitters', 'onshore_oil_gas_prod', 'gathering_boosting',
                   'transmission_pipelines', 'ldc_direct_emissions', 'sf6_from_elec_equip']
tot_em_columns = []

q_dict = {}

for t in emission_tables:
    qres = engine.execute(f"describe epa_ghgrp.{t}")
    tr = qres.fetchall()
    total_emission_cname = tr[-2][0]
    tot_em_columns.append(total_emission_cname)
    qres = engine.execute(f"select year, sum({total_emission_cname}) from epa_ghgrp.{t} group by year")
    q_dict[t] = pd.DataFrame(qres.fetchall(), columns=['year', total_emission_cname])

# A function that excludes terms using SQL to say "and X!=Y"
def excl_text(excl):
    return ' and '.join([f"latest_reported_industry_type_sectors!='{e}'" for e in excl])

# A function that includes text that matches; SQL that says "or X like '%Y%'"
def incl_text(excl):
    return ' or '.join([f"latest_reported_industry_type_sectors like '%{e}%'" for e in excl])

t = 'direct_emitters'
qres = engine.execute(f"describe epa_ghgrp.{t}")
t_cols = qres.fetchall()
total_emission_cname = t_cols[-2][0]

incl = [ 'Power', 'Petroleum']
qres = engine.execute(f"""
select year, sum({total_emission_cname}) from epa_ghgrp.{t}
where {incl_text(incl)}
group by year
""")
q_dict[t + f" (incl {','.join(incl)})"] = pd.DataFrame(qres.fetchall(), columns=['year', total_emission_cname + f" (matching {','.join(incl)})"])

excl = [ 'Minerals', 'Other', 'Waste', 'Chemicals', 'Pulp and Paper,Waste',
        'Metals,Waste', 'Pulp and Paper']
qres = engine.execute(f"""
select year, sum({total_emission_cname}) from epa_ghgrp.{t}
where {excl_text(excl)}
group by year
""")
q_dict[t + f" (excl {','.join(excl)})"] = pd.DataFrame(qres.fetchall(), columns=['year', total_emission_cname + f" (excl {','.join(excl)})"])

for t in emission_tables:
    qres = engine.execute(f"describe epa_ghgrp.{t}")
    tr = qres.fetchall()
    total_emission_cname = tr[-2][0]
    qres = engine.execute(f"select year, sum({total_emission_cname}) from epa_ghgrp.{t} group by year")
    q_dict[t] = pd.DataFrame(qres.fetchall(), columns=['year', total_emission_cname])

grand_total = {}

for year in q_dict['direct_emitters'].year:
    grand_total[year] = sum([q_dict[t][q_dict[t].year==year].iat[0,1] for t in emission_tables if year in q_dict[t].year.values])

df = pd.DataFrame.from_dict(grand_total, orient='index', columns=['total_co2e']).reset_index()
df.rename(columns={'index':'year'}, inplace=True)
q_dict['grand_total'] = df

This gem comes from https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes

In [ ]:
from functools import reduce

df_merged = reduce(lambda left,right: pd.merge(left,right,on=['year'], how='outer'), q_dict.values()).fillna(0)
df_merged.sort_values(by='year', ascending=False, inplace=True)
df_merged.index = pd.RangeIndex(len(df_merged.index))

A summary table consolidating the totals from the GHGRP, plus three additional columns:
1. direct emitters that match "Power" or "Petroleum"
2. direct emitters that are not the top other industries
3. total co2e

In [ ]:
df_merged.rename(columns={v:v.replace('_', ' ') for v in df_merged.columns.values})

# Cross-check with ESSD tops-down dataset

A quick look at *just* CO2.  We'll look at CO2e in the next set of cells.

In [ ]:
qres = engine.execute("""
select format('%tY', year), sector_title, format('%,.2f', sum(value)/1000000000) || ' GtCO2' as GtCO2 from essd.ghg_data
where sector_title='Energy systems' and gas='CO2' and year>DATE('2010-01-01') and year<DATE('2020-01-01') and ISO='USA'
group by year, sector_title, gas order by year desc""")
qres.fetchall()

In [ ]:
qres = engine.execute('describe essd.ghg_data')
qres.fetchall()

In [ ]:
qres = engine.execute('describe essd.gwp100_data')
qres.fetchall()

A look at CO2e (presuming that's what GHG gives us from the GWP100 table) for the category `Energy Systems`.

In [ ]:
qres = engine.execute("""
select format('%tY', year), sector_title, format('%,.2f', sum(GHG)/1000000000) || ' GtCO2' as GtCO2 from essd.gwp100_data
where sector_title='Energy systems' and year>DATE('2010-01-01') and year<DATE('2020-01-01') and ISO='USA'
group by year, sector_title order by year desc""")
qres.fetchall()

# Connect with economic data provided by US CENSUS All-sector Survey (2017)

In [ ]:
qres = engine.execute("describe us_census.all_sector_survey_2017")
display(qres.fetchall())
qres = engine.execute("select * from us_census.all_sector_survey_2017 where naics2012='221112'")
display(qres.fetchall())


Exercise the connection to NAICS and sector information provided by US Department of Commerce (US_CENSUS)

In [ ]:
# Show how many facilities are tagged with what primary NAICS codes

qres = engine.execute(f"""
select count (*), format('%tY', epa_ghgrp.direct_emitters.year), primary_naics_code, naics2012_label
from epa_ghgrp.direct_emitters, us_census.all_sector_survey_2017
where primary_naics_code=naics2012
      and us_census.all_sector_survey_2017.year='2017' and epa_ghgrp.direct_emitters.year=DATE('2017-01-01')
group by epa_ghgrp.direct_emitters.year, primary_naics_code, naics2012_label
order by count (*) desc limit 20
""")
display(qres.fetchall())

# More table reshaping: attribution estimation

In [ ]:
df = pd.read_sql("""
select facility_id, year, latitude, longitude, latest_reported_industry_type_sectors, total_reported_direct_emissions
from epa_ghgrp.direct_emitters""", engine)
df.facility_id = df.facility_id.astype('int64')
df.year = df.year.astype('datetime64[ns, UTC]')
df.total_reported_direct_emissions = df.total_reported_direct_emissions.astype('float64')
df.latest_reported_industry_type_sectors.fillna('Other', inplace=True)

df['sector_groupings'] = pd.Series([f"{s[0]} ({len(s)+1})" if len(s)>1 else s[0] for s in df.latest_reported_industry_type_sectors.str.split(',')])

In [ ]:
for sl in df.latest_reported_industry_type_sectors.str.split(','):
    # Ensure all primary (and if listed, secondary) sectors are represented
    if f's_{sl[0]}' not in df.columns:
        df[f's_{sl[0]}'] = 0.0
    if len(sl)>1 and f's_{sl[1]}' not in df.columns:
        df[f's_{sl[1]}'] = 0.0

In [ ]:
attribution_vector = [ pd.Series([1.0]),
                       pd.Series([2.0/3.0, 1.0/3.0]),
                       pd.Series([0.5, 0.3, 0.2]),
                       pd.Series([0.4, 0.3, 0.2, 0.1]),
                       pd.Series([0.30, 0.25, 0.20, 0.15, 0.10]),
                       pd.Series([0.30, 0.24, 0.19, 0.14, 0.09, 0.04])]

def apply_attribution(x):
    sl = x.latest_reported_industry_type_sectors.split(',')
    # Tertiary sectors not previously mentioned are silently converted to Other, keeping our attribution columns from exploding
    appropriate_columns = set([f's_{s}' if f's_{s}' in x else 's_Other' for s in sl])
    x[ appropriate_columns ] = x.total_reported_direct_emissions * attribution_vector[len(appropriate_columns)-1].values
    return x

df_emitters = df.apply(apply_attribution, axis=1)

In [ ]:
df_emitters[df_emitters.latest_reported_industry_type_sectors.str.contains(',')]

In [ ]:
df_emitters[df_emitters.latest_reported_industry_type_sectors.str.count(',')>1]

# Working with Materialized Views

Here's an example of a facility with many owners...

In [ ]:
qres = engine.execute("""select ghgrp_facility_id,frs_id_facility,lei,format('%tY', reporting_year),facility_name,facility_city,facility_state,parent_company_name,facility_naics_code
from epa_ghgrp.parent_company where reporting_year=DATE('2020-01-01') and ghgrp_facility_id=1005071 order by lei""")
qres.fetchall()

...meaning 10 rows of data that's outside our easy-to-aggregate data

In [ ]:
qres = engine.execute("""
select facility_id,facility_name,total_reported_direct_emissions,city,state,latitude,longitude,primary_naics_code,
       latest_reported_industry_type_subparts,latest_reported_industry_type_sectors,format('%tY', year)
from epa_ghgrp.direct_emitters where facility_id=1005071 order by year""")
qres.fetchall()

Create actual materialized view from a large concatenation operation

In [ ]:
import osc_ingest_trino as osc

engine.execute("create schema if not exists ghgrp_demo")

# display([(x, y) for x, y in zip(emission_tables,tot_em_columns)])

emission_selects = [ f"""
select ghgrp_facility_id, reporting_year, lei, '{e_tbl}' as table_source,
         primary_naics_code, parent_co_percent_ownership * 0.01 * {e_col} as fractional_emissions,
         facility_naics_code, parent_company_name
    from epa_ghgrp.parent_company as PC join epa_ghgrp.{e_tbl} as ET on PC.ghgrp_facility_id=ET.facility_id and PC.reporting_year=ET.year
""" for e_tbl, e_col in zip(emission_tables,tot_em_columns) ]

qres = engine.execute("drop table if exists ghgrp_demo.parent_attribution")
print(qres.fetchall())

sql = f"""
create table ghgrp_demo.parent_attribution as {' union all '.join(emission_selects)}
"""

print(sql)

qres = engine.execute(sql)
print(qres.fetchall())

In [ ]:
qres = engine.execute('describe ghgrp_demo.parent_attribution')
display(qres.fetchall())

qres = engine.execute("""
select ghgrp_facility_id, format('%tY', reporting_year), lei, table_source, format('%,.2f', fractional_emissions) || ' t CO2e' as metric
from ghgrp_demo.parent_attribution""")
qres.fetchall()[::2000]

How many **_facilities owned by public companies_** match to corporate reports we can see using the SEC's DERA dataset?

See how many `PARENT_COMPANY` records have LEIs we know.  Note that there are about 8400 total facilities, so 4 facilities not covered by LEI for each that is.
There are 3K-4K distinctly named entities, so average entity owns (at least partially) approx 2-3 facilities.  It also means we know the LEIs of approximately half of the parent copmanies.

In [ ]:
qres = engine.execute(f"""select count (*), format('%tY', reporting_year)
from (select lei, reporting_year from epa_ghgrp.parent_company where LEI is not null group by lei, reporting_year)
group by reporting_year order by reporting_year desc""")
qres.fetchall()

In [ ]:
qres = engine.execute(f"describe sec_dera.sub")
qres.fetchall()

In [ ]:
qres = engine.execute(f"describe sec_dera.num")
qres.fetchall()

In [ ]:
qres = engine.execute(f"""select count (*), format('%tY', reporting_year)
from ghgrp_demo.parent_attribution, sec_dera.sub
where (form='10-K' or form='20-F')
and reporting_year >= DATE('2020-01-01') and filed >= DATE('2020-01-01') and filed < DATE('2021-01-01')
and ghgrp_demo.parent_attribution.lei=sec_dera.sub.lei and ghgrp_demo.parent_attribution.lei is not null
group by ghgrp_demo.parent_attribution.reporting_year
order by ghgrp_demo.parent_attribution.reporting_year
""")
qres.fetchall()

We can tie these companies to ticker symbols...

In [ ]:
qres = engine.execute(f"""select * from sec_dera.ticker limit 10""")
qres.fetchall()

How many distinct companies own these facilities (and what are their ticker symbols)?

In [ ]:
qres = engine.execute(f"""
with leis as (select DISTINCT(sec_dera.sub.lei), name, if(tname IS NULL, '<private>', tname) as ticker
              from ghgrp_demo.parent_attribution,sec_dera.sub,sec_dera.ticker
              where ghgrp_demo.parent_attribution.lei=sec_dera.sub.lei
              and (form='10-K' or form='20-F')
              and sec_dera.sub.cik=sec_dera.ticker.cik
              and period>=DATE('2020-01-01')
              and period<DATE('2021-01-01'))
select count (*), ticker, leis.lei, name, format('%tY', reporting_year)
from ghgrp_demo.parent_attribution, leis
where reporting_year >= DATE('2020-01-01')
and ghgrp_demo.parent_attribution.lei=leis.lei
group by leis.ticker, leis.lei, name, reporting_year
order by count(*) desc
-- limit 10
""")
ticker_list = qres.fetchall()
print(len(ticker_list))

Note that some comapnies have more than one ticker symbol!

In [ ]:
ticker_list[0:50]

We can try to add up all the faciltiies for all the tickers, but that leads to counting duplicates for companies that have multiple ticker symbols...(should be 2651, not 5746)

In [ ]:
sum([te[0] for te in ticker_list])

Sample data to cross-check LEI, Facility ID and EDGAR submission data

In [ ]:
qres = engine.execute(f"""
select DISTINCT(sec_dera.sub.lei), ghgrp_facility_id, adsh
              from ghgrp_demo.parent_attribution,sec_dera.sub
              where reporting_year>=DATE('2020-01-01')
              and ghgrp_demo.parent_attribution.lei=sec_dera.sub.lei
              and (form='10-K' or form='20-F')
              and period>=DATE('2020-01-01') and period<DATE('2021-01-01')
              order by sec_dera.sub.lei desc""")
l = qres.fetchall()
print(len(l))
display(l[::100])

Compute intensity in metric tons of CO2e per million dollars

In [ ]:
qres = engine.execute(f"""
select PA.lei, sic, floor(sic/100) as sic_2digit, format('%1$tY-%1$tm-%1$td', reporting_year),
       name, sum(fractional_emissions) as tot_co2e,
       uom || ' $M', round(max(value)/1000000,3) as tot_revenue,
       format('%7.2f', 1000000*sum(fractional_emissions)/sum(value)) || ' tCO2e/$M' as intensity
from ghgrp_demo.parent_attribution as PA join sec_dera.sub as S on PA.lei=S.lei
     join sec_dera.num as N on S.adsh=N.adsh
where reporting_year>=DATE('2020-01-01')
and (form='10-K' or form='20-F')
and period>=DATE('2020-01-01') and period<DATE('2021-01-01')
and ddate>=DATE('2020-01-01') and ddate<DATE('2021-01-01')
and coreg is NULL
and (N.tag='Revenues'
     or N.tag='RevenueFromContractWithCustomerIncludingAssessedTax'
     or N.tag='RevenueFromContractWithCustomerExcludingAssessedTax'
     or N.tag='RevenuesNetOfInterestExpense'
     or N.tag='RegulatedAndUnregulatedOperatingRevenue'
     or N.tag='RegulatedOperatingRevenuePipelines')
and N.qtrs=4
group by PA.lei, PA.reporting_year, sic, name, uom
order by intensity desc
-- limit 100
""")
list = qres.fetchall()
print(len(list))
display(list[::5])

# A Deep Dive into outlier data

In [ ]:
qreg=engine.execute("""select DISTINCT(sec_dera.sub.lei), ghgrp_facility_id, name, adsh
              from ghgrp_demo.parent_attribution,sec_dera.sub
              where reporting_year>=DATE('2020-01-01') and sec_dera.sub.lei='549300O4B5CVWMKUES27'
              and ghgrp_demo.parent_attribution.lei=sec_dera.sub.lei
              and (form='10-K' or form='20-F')
              and period>=DATE('2020-01-01') and period<DATE('2021-01-01')""")
qreg.fetchall()

In [ ]:
qreg=engine.execute("""select reporting_year, format ('%,10.2f', sum(fractional_emissions)) || ' t CO2e' as metric
              from ghgrp_demo.parent_attribution
              where lei='549300O4B5CVWMKUES27'
              group by reporting_year
""")
l = qreg.fetchall()
l

# GHGRP Direct Emitters include Cement and Steel Plans (which we can connect to SFI data)

In [ ]:
qres = engine.execute("describe sfi_geoasset.cement")
display(qres.fetchall())

In [ ]:
qres = engine.execute("select count (*) from sfi_geoasset.cement")
display(qres.fetchall())
qres = engine.execute("select count (*) from sfi_geoasset.steel")
display(qres.fetchall())

# There are 105 US-located cement plants listed in the SFI report with parent LEIs
qres = engine.execute("select count (*), iso3 from sfi_geoasset.cement where iso3='USA' group by iso3")
display(qres.fetchall())

qres = engine.execute("""
select owner_name, parent_name, lei, parent_lei, facility_id
from sfi_geoasset.cement, epa_ghgrp.direct_emitters, epa_ghgrp.parent_company
where ghgrp_facility_id=facility_id
and reporting_year=epa_ghgrp.direct_emitters.year
and reporting_year>=DATE('2019-01-01') and reporting_year<DATE('2020-01-01')
and sfi_geoasset.cement.iso3='USA'
and abs(sfi_geoasset.cement.latitude-epa_ghgrp.direct_emitters.latitude)<0.01
and abs(sfi_geoasset.cement.longitude-epa_ghgrp.direct_emitters.longitude)<0.01
""")
l = qres.fetchall()
print(f"{len(l)}: facilities/parent relationships matched in USA using lat/lon")

In [ ]:
l[3::2]